# ETL DEL DATASET USER ITEMS

##### Importar librerías

In [5]:
#proporciona funcionalidades para trabajar con archivos comprimidos en el formato gzip (.gz)
import gzip

from pandas import json_normalize

#proporciona herramientas para trabajar con datos en formato JSON
import json 

#se usará para para evaluar cada línea del archivo como una expresión de Python
#lo que permite cargar datos que no están en formato JSON.
import ast 

import numpy as np
import pandas as pd


##### 1. Cargar y descomprimir el dataset

<p>Ahora, para abrir el archivo comprimido utilizaremos un bucle -for- que almacena la data en una lista vacía en la variable "data"</p>


In [6]:
data = []
with gzip.open('users_items.json.gz', 'rb') as f:
    for line in f:
        data.append(ast.literal_eval(line.decode('utf-8')))
        # *  = (json.loads(line)) 
        # ** = (ast.literal_eval(line.decode('utf-8'))) esta opcion porque no es json 
users_items = pd.DataFrame(data)
f.close()

In [7]:
#df_respaldo = users_items

##### 2. Explorar el dataset

Exploramos un poco el dataset y se hace evidente que la columna items contiene datos semiestructurados que deben ser desanidados y cada uno de esos elementos, debe tener su propia columna. 

In [8]:
users_items

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."
...,...,...,...,...,...
88305,76561198323066619,22,76561198323066619,http://steamcommunity.com/profiles/76561198323...,"[{'item_id': '413850', 'item_name': 'CS:GO Pla..."
88306,76561198326700687,177,76561198326700687,http://steamcommunity.com/profiles/76561198326...,"[{'item_id': '11020', 'item_name': 'TrackMania..."
88307,XxLaughingJackClown77xX,0,76561198328759259,http://steamcommunity.com/id/XxLaughingJackClo...,[]
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"[{'item_id': '304930', 'item_name': 'Unturned'..."


In [9]:
#Examinamos a nivel general cómo se encuentra compuesto el dataset
users_items.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88310 entries, 0 to 88309
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      88310 non-null  object
 1   items_count  88310 non-null  int64 
 2   steam_id     88310 non-null  object
 3   user_url     88310 non-null  object
 4   items        88310 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.4+ MB


##### 3. Limpiar y desanidar columnas del dataset

In [10]:
#Visualizamos los diccionarios anidados en la columna items. Hacemos un slice a la lista.
users_items["items"][0][:20]

[{'item_id': '10',
  'item_name': 'Counter-Strike',
  'playtime_forever': 6,
  'playtime_2weeks': 0},
 {'item_id': '20',
  'item_name': 'Team Fortress Classic',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '30',
  'item_name': 'Day of Defeat',
  'playtime_forever': 7,
  'playtime_2weeks': 0},
 {'item_id': '40',
  'item_name': 'Deathmatch Classic',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '50',
  'item_name': 'Half-Life: Opposing Force',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '60',
  'item_name': 'Ricochet',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '70',
  'item_name': 'Half-Life',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '130',
  'item_name': 'Half-Life: Blue Shift',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '300',
  'item_name': 'Day of Defeat: Source',
  'playtime_forever': 4733,
  'playtime_2weeks': 0},
 {'item_id': '240',
  'item_name': 'Counter-Strike: S

In [11]:
# Desanidamos la columna 'items'
users_items = pd.json_normalize(data, record_path=['items'], meta=['steam_id','items_count','user_id','user_url'] )

In [27]:
users_items

,item_id,item_name,playtime_forever,playtime_2weeks,steam_id,items_count,user_id,user_url,hours_game
0,10,Counter-Strike,6,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,0.10
1,20,Team Fortress Classic,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,0.00
2,30,Day of Defeat,7,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,0.12
3,40,Deathmatch Classic,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,0.00
4,50,Half-Life: Opposing Force,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,0.00
...,...,...,...,...,...,...,...,...,...
5153204,346330,BrainBread 2,0,0,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,0.00
5153205,373330,All Is Dust,0,0,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,0.00
5153206,388490,One Way To Die: Steam Edition,3,3,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,0.05
5153207,521570,You Have 10 Seconds 2,4,4,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,0.07


In [25]:
users_items.info()


<class 'pandas.core.frame.DataFrame'>
Index: 5094105 entries, 0 to 5153208
Data columns (total 9 columns):
 #   Column            Dtype  
---  ------            -----  
 0   item_id           object 
 1   item_name         object 
 2   playtime_forever  int64  
 3   playtime_2weeks   int64  
 4   steam_id          object 
 5   items_count       object 
 6   user_id           object 
 7   user_url          object 
 8   hours_game        float64
dtypes: float64(1), int64(2), object(6)
memory usage: 388.6+ MB


In [14]:
# Buscamos registros duplicados
duplicados = users_items[users_items.duplicated(subset=['steam_id','item_id','playtime_forever'])]
duplicados.shape

(59117, 8)

In [15]:
#Borramos dulplicados
users_items = users_items.drop_duplicates(keep='first')

In [36]:
# Convertir la columna 'playtime_forever' de minutos a horas
users_items['hours_game'] = users_items['playtime_forever'] / 60

# Redondear los valores a dos decimales (opcional)
users_items['hours_game'] = users_items['hours_game'].round(2)

In [37]:
users_items.head(100)

,item_id,item_name,playtime_forever,playtime_2weeks,steam_id,items_count,user_id,user_url,hours_game
0,10,Counter-Strike,6,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,0.10
1,20,Team Fortress Classic,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,0.00
2,30,Day of Defeat,7,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,0.12
3,40,Deathmatch Classic,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,0.00
4,50,Half-Life: Opposing Force,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,0.00
...,...,...,...,...,...,...,...,...,...
95,33930,Arma 2: Operation Arrowhead,3,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,0.05
96,219540,Arma 2: Operation Arrowhead Beta (Obsolete),0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,0.00
97,17410,Mirror's Edge,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,0.00
98,205790,Dota 2 Test,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,0.00


In [38]:
# Instanciamos el dataframe users_items 
df_users_items = users_items

In [39]:
# Obtenemos el top 5 valores máximos
top_5_max = df_users_items.nlargest(5, 'hours_game')

# Obtenemos el top 5 valores mínimos
top_5_min = df_users_items.nsmallest(5, 'hours_game')

# Mostrar los resultados
print("Top 5 valores máximos:")
print(top_5_max[['hours_game']])

print("\nTop 5 valores mínimos:")
print(top_5_min[['hours_game']])

Top 5 valores máximos:
         hours_game
587715     10712.88
2499068    10588.25
4075729    10540.87
1495340    10223.52
1836985    10001.13

Top 5 valores mínimos:
   hours_game
1         0.0
3         0.0
4         0.0
5         0.0
6         0.0


In [41]:

# Contar las filas con valor 0 en la columna 'hours_game'
filas_con_cero = (df_users_items['hours_game'] == 0).sum()

# Imprimir el resultado
print(f"Existen {filas_con_cero} filas con valor 0 en la columna 'hours_game'.")

Existen 1847730 filas con valor 0 en la columna 'hours_game'.


In [44]:
# Eliminar las filas con valor 0 en la columna 'hours_game'
df_users_items = df_users_items[df_users_items['hours_game'] != 0]

1,847,730 registros con valor "0" se eliminaron de la columna 'hours_game'. Esta decisión se toma teniendo en cuenta que una de las consultas requeridas es la de "def UserForGenre"

In [51]:
#Verificamos que las filas de "hours_game" = a 0 hayan sido eliminadas
df_users_items["hours_game"].min()

0.02

In [52]:
df_users_items.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3246375 entries, 0 to 5153208
Data columns (total 9 columns):
 #   Column            Dtype  
---  ------            -----  
 0   item_id           object 
 1   item_name         object 
 2   playtime_forever  int64  
 3   playtime_2weeks   int64  
 4   steam_id          object 
 5   items_count       object 
 6   user_id           object 
 7   user_url          object 
 8   hours_game        float64
dtypes: float64(1), int64(2), object(6)
memory usage: 247.7+ MB


In [53]:
#Eliminamos las columnas que no vamos a necesitar
df_users_items = df_users_items.drop(['playtime_2weeks','user_url','items_count','steam_id','playtime_forever'], axis=1)
df_users_items.columns

Index(['item_id', 'item_name', 'user_id', 'hours_game'], dtype='object')

In [54]:
df_users_items

,item_id,item_name,user_id,hours_game
0,10,Counter-Strike,76561197970982479,0.10
2,30,Day of Defeat,76561197970982479,0.12
8,300,Day of Defeat: Source,76561197970982479,78.88
9,240,Counter-Strike: Source,76561197970982479,30.88
10,3830,Psychonauts,76561197970982479,5.55
...,...,...,...,...
5153202,304930,Unturned,76561198329548331,11.28
5153203,227940,Heroes & Generals,76561198329548331,0.72
5153206,388490,One Way To Die: Steam Edition,76561198329548331,0.05
5153207,521570,You Have 10 Seconds 2,76561198329548331,0.07


### 4. Guardar el dataset limpio
Exportamos a CSV/JSON/PARQUET para tener varias opciones de archivo y minimizar reprocesos en el despliegue.

In [57]:
# Los archivos se almacenan en local 
#df_users_items.to_csv(r'data\users_items_cln.csv', index=False)
#df_users_items.to_json(r'data\users_items_cln.json', orient='records', lines=True)
#df_users_items.to_parquet(r'data\users_items_cln.parquet', index=False)